<a href="https://colab.research.google.com/github/RICQW/pdf-input-url/blob/main/clean_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install camelot-py

In [22]:
import camelot

# 读取 PDF 表格
tables = camelot.read_pdf("https://cdn.jsdelivr.net/gh/RICQW/pdf-input-url/permitapplication(2).pdf", pages="all")

# 初始化 result
result = []

row_counter = 1  # 连续行编号
num_cols = 6     # 根据你的表格列数修改

# 1️⃣ 读取并清理内容
for table in tables:
    df = table.df
    for r in range(len(df)):
        for c in range(len(df.columns)):
            cleaned = df.iloc[r, c].replace(" ", "").replace("\n", "")
            cell_id = f"R{row_counter}C{c+1}"
            result.append((cell_id, cleaned))
        row_counter += 1

# 2️⃣ 按行分组
rows = [result[i:i+num_cols] for i in range(0, len(result), num_cols)]

# 3️⃣ 处理序号行：合并下一行内容到上一行
i = 1  # 从第二行开始，跳过第一行
while i < len(rows) - 1:  # 保证有下一行
    current_row = rows[i]
    if any(cell[1] == "序号" for cell in current_row):
        prev_row = rows[i-1]
        next_row = rows[i+1]
        for j in range(len(current_row)):
            prev_row[j] = (prev_row[j][0], prev_row[j][1] + next_row[j][1])
        # 删除序号行和下一行
        rows.pop(i+1)
        rows.pop(i)
        continue  # 不增加 i，下一行已经移位
    i += 1

# 4️⃣ 重新编号
final_result = []
row_counter = 1
for row in rows:
    for c_idx, cell in enumerate(row):
        cell_id = f"R{row_counter}C{c_idx+1}"
        final_result.append((cell_id, cell[1]))
    row_counter += 1

# 5️⃣ 输出最终结果
print(final_result)


[('R1C1', '序号'), ('R1C2', '类别'), ('R1C3', '记录内容'), ('R1C4', '记录频次'), ('R1C5', '记录形式'), ('R1C6', '其他信息'), ('R2C1', '1'), ('R2C2', '基本信息'), ('R2C3', '基本信息主要包括企业名称、生产经营场所地址、行业类别、法定代表人、统一社会信用代码、产品名称、生产工艺、生产规模、环保投资、排污权交易文件、环境影响评价审批意见及排污许可证编号等。'), ('R2C4', '对于未发生变化的基本信息，按年记录，1次/a；对于发生变化的基本信息，在发生变化时记录。'), ('R2C5', '电子台账+纸质台账'), ('R2C6', '台账保存期限不得少于五年。'), ('R3C1', '2'), ('R3C2', '监测记录信息'), ('R3C3', '对于采用手工监测的工业噪声排污单位，应记录手工监测时段信息、噪声污染防治设施维修和更换情况。手工监测时段信息应记录监测时段内非正常工况情形、事件原因、是否报告、应对措施等；监测时段内工业噪声排放值超标情况，包括超标原因、是否报告、应对措施等。'), ('R3C4', '每发生一次记录1次。'), ('R3C5', '电子台账+纸质台账'), ('R3C6', '台账保存期不得少于5年。'), ('R4C1', '3'), ('R4C2', '监测记录信息'), ('R4C3', '生产运行情况包括生产设施、公用单元和全厂运行情况，重点记录排污许可证中相关信息的实际情况及与污染物治理、排放相关的主要运行参数。正常工况各生产单元主要生产设施的累计生产时间，生产实际负荷，主要产品产量，原辅材料及燃料使用情况'), ('R4C4', 'a）正常工况：1）运行状态：按照各生产单元生产班制记录，每班记录1次。'), ('R4C5', '电子台账+纸质台账'), ('R4C6', '台账保存期限不得少于五年。'), ('R5C1', '序号'), ('R5C2', '类别'), ('R5C3', '记录内容'), ('R5C4', '记录频次'), ('R5C5', '记录形式'), ('R5C6', '其他信息'), ('R6C1', ''), ('R6C2', ''), ('R6C3', '等数据。生

In [21]:
import csv

with open("ledger(2).csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerows(final_result)
